# Конструирование признаков (Feature Engineering)

---

**Источники:**

[Искусство Feature Engineering в машинном обучении](https://habr.com/ru/company/mlclass/blog/248129/)

[Конструирование признаков](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D0%BD%D1%81%D1%82%D1%80%D1%83%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BF%D1%80%D0%B8%D0%B7%D0%BD%D0%B0%D0%BA%D0%BE%D0%B2)

[Representation: Feature Engineering](https://developers.google.com/machine-learning/crash-course/representation/feature-engineering)

[7 Feature Engineering Techniques in Machine Learning You Should Know](https://www.analyticsvidhya.com/blog/2020/10/7-feature-engineering-techniques-machine-learning/)

[Feature Engineering — deep dive into Encoding and Binning techniques](https://towardsdatascience.com/feature-engineering-deep-dive-into-encoding-and-binning-techniques-5618d55a6b38)

[Feature Engineering in Machine Learning](https://lucasxlu.github.io/blog/2018/08/20/ml-feml/)

[Binning Data with Pandas qcut and cut](https://pbpython.com/pandas-qcut-cut.html)

[Data Preprocessing with Python Pandas — Part 5 Binning](https://towardsdatascience.com/data-preprocessing-with-python-pandas-part-5-binning-c5bd5fd1b950)

---

## Подготовка окружения

In [ ]:
# ВНИМАНИЕ: необходимо удостовериться, что виртуальная среда выбрана правильно!

# Для MacOS/Ubuntu
# !which pip

# Для Windows
# !where pip

In [ ]:
# !conda install matplotlib numpy missingno scikit-learn -y

In [ ]:
# !conda install -c conda-forge category_encoders -y

In [4]:
import numpy as np

np.__version__

'1.19.2'

In [5]:
import pandas as pd

pd.__version__

'1.2.3'

## Создание `DataFrame` для примера

In [6]:
# товары в интернет магазине
df = pd.DataFrame({'user_name': pd.Series(['Mr Oleg', 'Mr Oleg',
                                           'Mr Pete', 'Mr Pete', 
                                           'Mrs Elena', 'Mr Viktor',
                                           'Mr Anton', 'Mr Anton', 
                                           'Mrs Alex', 'Mrs Alex'], 
                                          dtype='string'),
                   
                  'user_rating': pd.Series(['5 star', '5 star', 
                                            '3 star', '3 star', 
                                            '1 star', '2 star', 
                                            '5 star', '5 star', 
                                            '4 star', '4 star'], 
                                           dtype=pd.CategoricalDtype(categories=['1 star', 
                                                                                 '2 star', 
                                                                                 '3 star', 
                                                                                 '4 star', 
                                                                                 '5 star'], ordered=True)),
                  'title': pd.Series(['toy', 'Beautiful Cat', 
                                      'Car', 'TV', 
                                      'Toys', 'Television', 
                                      'House', 'Dog', 
                                      'Kitty', 'Soft TOY'], 
                                     dtype='string'),
                   
                  
                   'tags': pd.Series(['children, beautiful, good_condition', 
                                      'animal, cat, beautiful', 
                                      'beautiful, good_condition', 
                                      'good_condition', 
                                      'children, animal', 
                                      'good_condition', 
                                      'good_condition, suburbs', 
                                      'animal, dog', 'animal', 
                                      'children, good_condition'], 
                                     dtype='string'),
                                     
                   'item_age_month': pd.Series(['0-1', '1-6', 
                                               '6-12', '12+', 
                                               '1-6', '1-6', 
                                               '12+', '0-1', 
                                               '6-12', '6-12'], 
                                              dtype=pd.CategoricalDtype(categories=['0-1', 
                                                                                    '1-6', 
                                                                                    '6-12', 
                                                                                    '12+'], ordered=True)),
                   
                   'color': pd.Series(['red', 'blue', 
                                           'rainbow', 'white', 
                                           'red', 'white', 
                                           'rainbow', 'white', 
                                           'white', 'blue'], 
                                          dtype='category'),
                   
                    'open_date_time': pd.Series(['10.10.2020 18:25', '20.12.2020 16:36', 
                                                 '06.04.2021 10:14', '07.04.2021 23:25', 
                                                 '25.11.2020 06:17', '16.12.2020 15:15', 
                                                 '10.03.2021 09:08', '31.12.2020 04:57', 
                                                 '28.12.2020 12:54', '29.12.2020 23:25'], dtype='datetime64[ns]'),
                   'status': pd.Series(['sold out', 'for_sale', 
                                        'for_sale', 'for_sale', 
                                        'sold out', 'sold out', 
                                        'for_sale', 'for_sale', 
                                        'sold out', 'for_sale'], 
                                       dtype='string')
                  })

In [7]:
# отображать все колонки
pd.options.display.max_columns = None

In [8]:
df

,user_name,user_rating,title,tags,item_age_month,color,open_date_time,status
0,Mr Oleg,5 star,toy,"children, beautiful, good_condition",0-1,red,2020-10-10 18:25:00,sold out
1,Mr Oleg,5 star,Beautiful Cat,"animal, cat, beautiful",1-6,blue,2020-12-20 16:36:00,for_sale
2,Mr Pete,3 star,Car,"beautiful, good_condition",6-12,rainbow,2021-06-04 10:14:00,for_sale
3,Mr Pete,3 star,TV,good_condition,12+,white,2021-07-04 23:25:00,for_sale
4,Mrs Elena,1 star,Toys,"children, animal",1-6,red,2020-11-25 06:17:00,sold out
5,Mr Viktor,2 star,Television,good_condition,1-6,white,2020-12-16 15:15:00,sold out
6,Mr Anton,5 star,House,"good_condition, suburbs",12+,rainbow,2021-10-03 09:08:00,for_sale
7,Mr Anton,5 star,Dog,"animal, dog",0-1,white,2020-12-31 04:57:00,for_sale
8,Mrs Alex,4 star,Kitty,animal,6-12,white,2020-12-28 12:54:00,sold out
9,Mrs Alex,4 star,Soft TOY,"children, good_condition",6-12,blue,2020-12-29 23:25:00,for_sale


Анализ 
`user_name` - номинальная шкала
`` - 
`` - 
`` - 
`` - 
`` - 
`` - 

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   user_name       10 non-null     string        
 1   user_rating     10 non-null     category      
 2   title           10 non-null     string        
 3   tags            10 non-null     string        
 4   item_age_month  10 non-null     category      
 5   color           10 non-null     category      
 6   open_date_time  10 non-null     datetime64[ns]
 7   status          10 non-null     string        
dtypes: category(3), datetime64[ns](1), string(4)
memory usage: 1.2 KB


In [10]:
# проверим правильно ли задан тип порядковой категории
df.item_age_month

0     0-1
1     1-6
2    6-12
3     12+
4     1-6
5     1-6
6     12+
7     0-1
8    6-12
9    6-12
Name: item_age_month, dtype: category
Categories (4, object): ['0-1' < '1-6' < '6-12' < '12+']

In [11]:
# в ячейках open_date_time содержаться данные типа Timestamp
df.open_date_time[0]

Timestamp('2020-10-10 18:25:00')

[Документация по pandas.Timestamp](https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html).

In [12]:
# у Timestamp можно получить номер дня в дате
df.open_date_time[0].day

10

In [13]:
# у Timestamp можно получить день года
df.open_date_time[0].day_of_year

284

## Конструирование признаков (Feature Engineering)

<blockquote>Придумывать признаки трудно, требует много времени и глубоких знаний. «Прикладное машинное обучение», в основном, это конструирование признаков.
— Эндрю Ын </blockquote>

**Признак** — это качество или свойство, присущее всем независимым объектам, анализ или прогноз для которых производится. Любое качество может быть полезным, если оно полезно для модели.

Смысл **признака** и чем он отличается от качества, легче было бы понять в контексте задачи. **Признак является характеристикой, которая может помочь решить задачу**.

**Признаки** в данных важны для используемых прогнозных моделей и влияют на результат, который необходимо получить. **Качество и количество признаков оказывают большое влияние на качество модели, будет она хорошей или нет.**

Можно сказать, что чем лучше **признаки**, тем лучше будет результат. Это не совсем полная правда, поскольку результат будет зависеть от модели и данных, не просто от выбранных **признаков**. Но отбор правильных **признаков** остаётся очень важным делом. Более качественные **признаки** могут дать более простую и более гибкую модель и часто они дают лучшие результаты.

**Конструирование признаков** — это процесс использования предметной области данных для создания признаков, которые нужны для машинного обучения. 

**Конструирование признаков** является фундаментом для приложений машинного обучения, а также трудным и затратным процессом. Необходимости ручного конструирования признаков можно избежать при автоматизации прикладного обучения признакам.

Почти любая задача начинается с создания (Engineering) и отбора (Selection) признаков.

**Процесс конструирования признаков** — это
- Метод мозгового штурма или проверка признаков;
- Решение, какие признаки создавать;
- Создание признаков;
- Проверка, какие признаки работают с моделью;
- Улучшение признаков, если требуется;
- Возврат к методу мозгового штурма/создание других признаков, пока работа не будет завершена.


Большинство алгоритмов (или библиотек машинного обучения) дают лучший результат с числовыми признаками.

В Python для библиотеки `sklearn` требуются признаки в числовых массивах (`np.array`, `pd.DataFrame`, `pd.Series`).

## Строковые признаки

**Возможное решение:**
- В самих строках зачастую содержится информация ("Mr.", "Mrs." преобразовать в половой признак).
- Сократить количество "категорий" ("кот", "кошка", "кошечка", "котенок" -> "кот").
- Разделить строку на несколько признаков (список тегов, ФИО), если их не очень много.
- Преобразовать к другому типу ("первое января 93-го года" преобразовать в 01.01.1993).
- ...

In [14]:
df.select_dtypes(include='string')

,user_name,title,tags,status
0,Mr Oleg,toy,"children, beautiful, good_condition",sold out
1,Mr Oleg,Beautiful Cat,"animal, cat, beautiful",for_sale
2,Mr Pete,Car,"beautiful, good_condition",for_sale
3,Mr Pete,TV,good_condition,for_sale
4,Mrs Elena,Toys,"children, animal",sold out
5,Mr Viktor,Television,good_condition,sold out
6,Mr Anton,House,"good_condition, suburbs",for_sale
7,Mr Anton,Dog,"animal, dog",for_sale
8,Mrs Alex,Kitty,animal,sold out
9,Mrs Alex,Soft TOY,"children, good_condition",for_sale


### Получить информацию из строки

ВНИМАНИЕ: Требуется понимание предметной области!

In [15]:
# предположим, что может быть только Mr или Mrs, тогда можно
# создать колонку (bool) определяющую пол пользователя
df['user_sex'] = df.user_name.str.contains("Mrs")

# False = Мужчина / True = Женщина
df.user_sex

0    False
1    False
2    False
3    False
4     True
5    False
6    False
7    False
8     True
9     True
Name: user_sex, dtype: boolean

### Сократить количество "категорий"

ВНИМАНИЕ: Требуется понимание предметной области!

*Может потребовать задействитвовать другие модели для получения похожих данных для группировки.

In [16]:
# какие наименования имеются?
df.title.value_counts()

Car              1
TV               1
Television       1
Dog              1
House            1
Kitty            1
Soft TOY         1
toy              1
Toys             1
Beautiful Cat    1
Name: title, dtype: Int64

In [17]:
# сформировать, что заменяем и на что
toy_replacement = dict.fromkeys(['toy', 'Toys', 'Soft TOY'], 'Toy')
toy_replacement

{'toy': 'Toy', 'Toys': 'Toy', 'Soft TOY': 'Toy'}

In [18]:
# сформировать, что заменяем и на что
cat_replacement = dict.fromkeys(['Beautiful Cat', 'Kitty'], 'Cat')
cat_replacement

{'Beautiful Cat': 'Cat', 'Kitty': 'Cat'}

In [19]:
# заменить данные в столбце
df.title.replace(to_replace=toy_replacement, inplace=True)
df.title.replace(to_replace=cat_replacement, inplace=True)
df.title.replace(to_replace='Television', value='TV', inplace=True)

In [20]:
df.title.value_counts()

Toy      3
TV       2
Cat      2
Car      1
Dog      1
House    1
Name: title, dtype: int64

### Разделить строку на несколько признаков

ВНИМАНИЕ: Требуется понимание предметной области!

In [21]:
df.tags

0    children, beautiful, good_condition
1                 animal, cat, beautiful
2              beautiful, good_condition
3                         good_condition
4                       children, animal
5                         good_condition
6                good_condition, suburbs
7                            animal, dog
8                                 animal
9               children, good_condition
Name: tags, dtype: string

In [22]:
# в параметр index попадает список тегов (разделенных), к ним добавляется префикс tag_
# метод apply применяется для каждой строки и возвращает DataFrame
tags_df = df.tags.apply(lambda x: pd.Series(True, index=[f'tag_{tag}' for tag in x.split(", ")]))
tags_df

,tag_children,tag_beautiful,tag_good_condition,tag_animal,tag_cat,tag_suburbs,tag_dog
0,True,True,True,NaN,NaN,NaN,NaN
1,NaN,True,NaN,True,True,NaN,NaN
2,NaN,True,True,NaN,NaN,NaN,NaN
3,NaN,NaN,True,NaN,NaN,NaN,NaN
4,True,NaN,NaN,True,NaN,NaN,NaN
5,NaN,NaN,True,NaN,NaN,NaN,NaN
6,NaN,NaN,True,NaN,NaN,True,NaN
7,NaN,NaN,NaN,True,NaN,NaN,True
8,NaN,NaN,NaN,True,NaN,NaN,NaN
9,True,NaN,True,NaN,NaN,NaN,NaN


In [23]:
# все незаполненные значения (NA) заменяются на False
tags_df = tags_df.fillna(False)
tags_df

,tag_children,tag_beautiful,tag_good_condition,tag_animal,tag_cat,tag_suburbs,tag_dog
0,True,True,True,False,False,False,False
1,False,True,False,True,True,False,False
2,False,True,True,False,False,False,False
3,False,False,True,False,False,False,False
4,True,False,False,True,False,False,False
5,False,False,True,False,False,False,False
6,False,False,True,False,False,True,False
7,False,False,False,True,False,False,True
8,False,False,False,True,False,False,False
9,True,False,True,False,False,False,False


In [24]:
# присоединить колонки с тегами к df
df = df.join(tags_df)

### Преобразовать к другому типу

ВНИМАНИЕ: Требуется понимание предметной области!

In [25]:
df.status

0    sold out
1    for_sale
2    for_sale
3    for_sale
4    sold out
5    sold out
6    for_sale
7    for_sale
8    sold out
9    for_sale
Name: status, dtype: string

In [26]:
df['is_sold_out'] = df.status.map({'sold out': True, 'for_sale': False})
df

,user_name,user_rating,title,tags,item_age_month,color,open_date_time,status,user_sex,tag_children,tag_beautiful,tag_good_condition,tag_animal,tag_cat,tag_suburbs,tag_dog,is_sold_out
0,Mr Oleg,5 star,Toy,"children, beautiful, good_condition",0-1,red,2020-10-10 18:25:00,sold out,False,True,True,True,False,False,False,False,True
1,Mr Oleg,5 star,Cat,"animal, cat, beautiful",1-6,blue,2020-12-20 16:36:00,for_sale,False,False,True,False,True,True,False,False,False
2,Mr Pete,3 star,Car,"beautiful, good_condition",6-12,rainbow,2021-06-04 10:14:00,for_sale,False,False,True,True,False,False,False,False,False
3,Mr Pete,3 star,TV,good_condition,12+,white,2021-07-04 23:25:00,for_sale,False,False,False,True,False,False,False,False,False
4,Mrs Elena,1 star,Toy,"children, animal",1-6,red,2020-11-25 06:17:00,sold out,True,True,False,False,True,False,False,False,True
5,Mr Viktor,2 star,TV,good_condition,1-6,white,2020-12-16 15:15:00,sold out,False,False,False,True,False,False,False,False,True
6,Mr Anton,5 star,House,"good_condition, suburbs",12+,rainbow,2021-10-03 09:08:00,for_sale,False,False,False,True,False,False,True,False,False
7,Mr Anton,5 star,Dog,"animal, dog",0-1,white,2020-12-31 04:57:00,for_sale,False,False,False,False,True,False,False,True,False
8,Mrs Alex,4 star,Cat,animal,6-12,white,2020-12-28 12:54:00,sold out,True,False,False,False,True,False,False,False,True
9,Mrs Alex,4 star,Toy,"children, good_condition",6-12,blue,2020-12-29 23:25:00,for_sale,True,True,False,True,False,False,False,False,False


## Категориальные признаки

**Пример:**
Цвет (color), т.е. синий (blue), красный (red), зеленый (green).


**Возможное решение:**
- Добавить комбинации признаков вида "is_red", "is_blue", "is_green", "is_red_or_blue" и т.п.
- Разбиение на интервалы (Binning = `cut`/`qcut`).
- Категория -> число (`map`, `replace`, `Dummies`, `OrdinalEncoder`, `LabelEncoder`, `OneHotEncoder`).
- Заменить признаки на их количество (`CountEncoder`).
- `TargetEncoder`.
- `CatBoostEncoder`.

### Комбинации признаков

In [27]:
df.color.value_counts()

white      4
blue       2
rainbow    2
red        2
Name: color, dtype: int64

In [28]:
df['is_white'] = df.color == 'white'
df['is_blue'] = df.color == 'blue'
df['is_rainbow'] = df.color == 'rainbow'
df['is_red'] = df.color == 'red'

In [29]:
df['is_color_white_or_blue'] = (df.color == 'white') | (df.color == 'blue')
df['is_color_rainbow_or_red'] = (df.color == 'rainbow') | (df.color == 'red')

In [30]:
df

,user_name,user_rating,title,tags,item_age_month,color,open_date_time,status,user_sex,tag_children,tag_beautiful,tag_good_condition,tag_animal,tag_cat,tag_suburbs,tag_dog,is_sold_out,is_white,is_blue,is_rainbow,is_red,is_color_white_or_blue,is_color_rainbow_or_red
0,Mr Oleg,5 star,Toy,"children, beautiful, good_condition",0-1,red,2020-10-10 18:25:00,sold out,False,True,True,True,False,False,False,False,True,False,False,False,True,False,True
1,Mr Oleg,5 star,Cat,"animal, cat, beautiful",1-6,blue,2020-12-20 16:36:00,for_sale,False,False,True,False,True,True,False,False,False,False,True,False,False,True,False
2,Mr Pete,3 star,Car,"beautiful, good_condition",6-12,rainbow,2021-06-04 10:14:00,for_sale,False,False,True,True,False,False,False,False,False,False,False,True,False,False,True
3,Mr Pete,3 star,TV,good_condition,12+,white,2021-07-04 23:25:00,for_sale,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False
4,Mrs Elena,1 star,Toy,"children, animal",1-6,red,2020-11-25 06:17:00,sold out,True,True,False,False,True,False,False,False,True,False,False,False,True,False,True
5,Mr Viktor,2 star,TV,good_condition,1-6,white,2020-12-16 15:15:00,sold out,False,False,False,True,False,False,False,False,True,True,False,False,False,True,False
6,Mr Anton,5 star,House,"good_condition, suburbs",12+,rainbow,2021-10-03 09:08:00,for_sale,False,False,False,True,False,False,True,False,False,False,False,True,False,False,True
7,Mr Anton,5 star,Dog,"animal, dog",0-1,white,2020-12-31 04:57:00,for_sale,False,False,False,False,True,False,False,True,False,True,False,False,False,True,False
8,Mrs Alex,4 star,Cat,animal,6-12,white,2020-12-28 12:54:00,sold out,True,False,False,False,True,False,False,False,True,True,False,False,False,True,False
9,Mrs Alex,4 star,Toy,"children, good_condition",6-12,blue,2020-12-29 23:25:00,for_sale,True,True,False,True,False,False,False,False,False,False,True,False,False,True,False


### Разбиение на интервалы (Binning)

### Replace

In [31]:
df.color.value_counts()

white      4
blue       2
rainbow    2
red        2
Name: color, dtype: int64

In [32]:
color_replacement = {name: num for num, name in enumerate(df.color.value_counts().index)}
color_replacement

{'white': 0, 'blue': 1, 'rainbow': 2, 'red': 3}

In [33]:
# ВНИМАНИЕ: не удаляет значения, которые не указаны в replace
df['color_nums'] = df.color.replace(color_replacement)
df.color_nums.value_counts()

0    4
1    2
2    2
3    2
Name: color_nums, dtype: int64

In [34]:
df

,user_name,user_rating,title,tags,item_age_month,color,open_date_time,status,user_sex,tag_children,tag_beautiful,tag_good_condition,tag_animal,tag_cat,tag_suburbs,tag_dog,is_sold_out,is_white,is_blue,is_rainbow,is_red,is_color_white_or_blue,is_color_rainbow_or_red,color_nums
0,Mr Oleg,5 star,Toy,"children, beautiful, good_condition",0-1,red,2020-10-10 18:25:00,sold out,False,True,True,True,False,False,False,False,True,False,False,False,True,False,True,3
1,Mr Oleg,5 star,Cat,"animal, cat, beautiful",1-6,blue,2020-12-20 16:36:00,for_sale,False,False,True,False,True,True,False,False,False,False,True,False,False,True,False,1
2,Mr Pete,3 star,Car,"beautiful, good_condition",6-12,rainbow,2021-06-04 10:14:00,for_sale,False,False,True,True,False,False,False,False,False,False,False,True,False,False,True,2
3,Mr Pete,3 star,TV,good_condition,12+,white,2021-07-04 23:25:00,for_sale,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,0
4,Mrs Elena,1 star,Toy,"children, animal",1-6,red,2020-11-25 06:17:00,sold out,True,True,False,False,True,False,False,False,True,False,False,False,True,False,True,3
5,Mr Viktor,2 star,TV,good_condition,1-6,white,2020-12-16 15:15:00,sold out,False,False,False,True,False,False,False,False,True,True,False,False,False,True,False,0
6,Mr Anton,5 star,House,"good_condition, suburbs",12+,rainbow,2021-10-03 09:08:00,for_sale,False,False,False,True,False,False,True,False,False,False,False,True,False,False,True,2
7,Mr Anton,5 star,Dog,"animal, dog",0-1,white,2020-12-31 04:57:00,for_sale,False,False,False,False,True,False,False,True,False,True,False,False,False,True,False,0
8,Mrs Alex,4 star,Cat,animal,6-12,white,2020-12-28 12:54:00,sold out,True,False,False,False,True,False,False,False,True,True,False,False,False,True,False,0
9,Mrs Alex,4 star,Toy,"children, good_condition",6-12,blue,2020-12-29 23:25:00,for_sale,True,True,False,True,False,False,False,False,False,False,True,False,False,True,False,1


### Map

### Dummies

- `pd.get_dummies` приводит к матрице `Pandas DataFrame`, тогда как `OneHotEncoder` приводит к матрице `SciPy CSR`.

- `pd.get_dummies` намного быстрее, чем `OneHotEncoder`.

- `OneHotEncoder` не может обрабатывать строковые значения напрямую. Если ваши номинальные характеристики представляют собой строки, вам необходимо сначала сопоставить их с целыми числами.

- `pandas.get_dummies` - полная противоположность. По умолчанию он преобразует только строковые столбцы в one-hot представление, если столбцы не указаны.

- Суть в том, что кодировщик `OneHotEncoder` создает функцию, которая сохраняется и затем может применяться к новым наборам данных, которые используют те же категориальные переменные, с согласованными результатами.

- С другой стороны, с `sklearn.OneHotEncoder`, после того как мы создали кодировщик, мы можем повторно использовать его для получения одного и того же вывода каждый раз, со столбцами только для "красного" и "зеленого".
Невозможно явно контролировать, что происходит, когда встречается новый "синий": если предполагается, что это невозможно, то можно указать, чтобы он выдал ошибку с помощью `handle_unknown = "error"`; в противном случае мы можем сказать ему продолжить и просто установить красный и зеленый столбцы в 0 с помощью `handle_unknown = "ignore"`.

In [35]:
dummies = pd.get_dummies(df[cat_features])
dummies

NameError: name 'cat_features' is not defined

In [ ]:
new_df = df_num.join(dummies)
new_df

### OrdinalEncoder

### LabelEncoder

Объект `LabelEncoder` присваивает каждому уникальному значению отдельное целое число.

<img src="images/label_encoder_example.png"/>

Этот подход **предполагает упорядочение категорий**:

`Никогда (0) < Редко (1) < Большинство дней (2) < Каждый день (3)`.

Это предположение имеет смысл в этом примере, потому что существует неоспоримое ранжирование категорий.

Не все категориальные переменные имеют четкий порядок значений, но мы называем те, которые имеют **порядковыми переменными**.

Для моделей на основе деревьев (таких, как деревья решений = `decision trees` и случайные леса = `random forests`) можно ожидать, что кодирование меток будет хорошо работать с порядковыми переменными.

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

encoded = df[cat_features].apply(encoder.fit_transform)
encoded

In [ ]:
new_df = df_num.join(encoded)
new_df

### OneHotEncoder

При использовании `OneHotEncoder` создаются новые столбцы, указывающие на наличие (или отсутствие) каждого возможного значения в исходных данных.

<img src="images/one_hot_encoder_example.png" />

В исходном наборе данных "Цвет" - это категориальная переменная с тремя категориями: "Красный", "Желтый" и "Зеленый".

Соответствующий `OneHotEncoder` содержит один столбец для каждого возможного значения и одну строку для каждой строки в исходном наборе данных.

Если исходным значением было "Красный", помещаем 1 в столбец "Красный"; если исходным значением было "Желтый", мы помещаем 1 в столбец "Желтый" и так далее.

В отличие от `LabelEncoder`, `OneHotEncoder` **не предполагает упорядочивания категорий**.

Таким образом, вы можете ожидать, что этот подход будет работать особенно хорошо, если в категориальных данных *нет четкого упорядочения* (например, "Красный" не больше и не меньше, чем "Желтый").

Категориальные переменные без внутреннего ранжирования называют **номинальными переменными**.

Объект `OneHotEncoder` обычно работает не очень хорошо, если категориальная переменная принимает большое количество значений (т.е. обычно не следует использовать его для переменных, принимающих более 15 различных значений).

In [ ]:
from sklearn.preprocessing import OneHotEncoder

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

OH_encoded = OH_encoder.fit_transform(df[cat_features])
OH_encoded

In [ ]:
column_names = OH_encoder.get_feature_names(cat_features)
column_names

In [ ]:
OH_encoded_with_names = pd.DataFrame(OH_encoded, columns=column_names)
OH_encoded_with_names

In [ ]:
new_df = df_num.join(OH_encoded_with_names)
new_df

### CountEncoder

Заменяет каждое категориальное значение количеством раз, сколько оно появляется в наборе данных.

Необходимо установить пакет [Category Encoders](http://contrib.scikit-learn.org/category_encoders/)

In [ ]:
!pip -V

In [ ]:
!conda install -c conda-forge category_encoders -y

In [ ]:
import category_encoders as ce

count_enc = ce.CountEncoder()

count_encoded = count_enc.fit_transform(df[cat_features])
count_encoded

In [ ]:
new_df = df_num.join(count_encoded.add_suffix("_count"))
new_df

### TargetEncoder

Заменяет категориальное значение средним значением для этого признака.

Например, учитывая дано значение для страны "RU", вычислите средний доход для всех строк с country == "RU", допустим 0.28.

Необходимо установить пакет [Category Encoders](http://contrib.scikit-learn.org/category_encoders/)

In [ ]:
import category_encoders as ce

target_enc = ce.TargetEncoder(cols=cat_features)

train_df = df.copy()

target_enc = target_enc.fit_transform(train_df[cat_features], train_df['CO2EMISSIONS'])
target_enc

In [ ]:
new_df = df_num.join(target_enc.add_suffix("_target"))
new_df

### CatBoostEncoder

Похож на `TargetEncoder` в том смысле, что оно основано на целевой вероятности (target probability) для данного значения. Однако с `CatBoostEncoder` для каждой строки целевая вероятность вычисляется только из строк перед ней.

Необходимо установить пакет [Category Encoders](http://contrib.scikit-learn.org/category_encoders/)

In [ ]:
import category_encoders as ce

target_enc = ce.CatBoostEncoder(cols=cat_features)

train_df = df.copy()

target_enc = target_enc.fit_transform(train_df[cat_features], train_df['CO2EMISSIONS'])
target_enc

In [ ]:
new_df = df_num.join(target_enc.add_suffix("_cb"))
new_df

## Числовые признаки

**Возможное решение:**
- Округление или разделение на целую и вещественную часть (+ нормализация).
- Приведение числового признака в категориальный (Binning). Пример: добавить признаки вида "рост больше X", "рост меньше X".

<img src="images/feature_engineering.png">

## Дата и время

**Возможное решение:**
- Добавить признаки, соответствующие 
    - времени дня
    - количеству прошедшего времени с определенного момента
    - выделение сезонов, времен года, кварталов, праздничных дней
    - состояние погоды для этих дней (может быть в дождливые дни поведение покупателей меняется?)
- Разделение времени на часы, минуты и секунды (если время дано в Unix-Time или ISO формате). 

## Результаты других алгоритмов

Если решается задача классификации, можно сначала решить вспомогательную задачу кластеризации, и в качестве признака в первоначальной задаче взять кластер объекта.

Это обычно происходит на основе первичного анализа данных в случае, когда объекты хорошо кластеризуются.

## Агрегированные признаки

Признаки, которые агрегируют признаки некоторого объекта, тем самым также сокращая размерность признакового описания.

Полезно в задачах, в которых один объект содержит несколько однотипных параметров.

Например, человек, имеющий несколько автомобилей разной стоимости. В данном случае можно рассмотреть признаки, соответствующие максимальной/минимальной/средней стоимости автомобиля этого человека.

## *Добавление новых признаков

Чтобы эффективно решить задачу, необходимо быть экспертом в конкретной области и понимать, что влияет на конкретную целевую переменную. 

## Рекомендации

- Избегайте редко используемых дискретных значений признаков.
    - Хорошие значения характеристик должны появляться в наборе данных более 5 раз.
    - И наоборот, если значение появляется только один раз или очень редко, модель не может делать прогнозы на основе этого значения признака.
- Стремитесь к ясным и очевидным значениям.
    - Каждый признак должен иметь ясное и очевидное значение для всех участников проекта. 
- Учитывайте нестабильность входящего потока.
    - Определение признака не должно менять со временем.
    - Но при получении значения с помощью другой модели сопряжено с дополнительными трудностями.
- Чаще всего необходимо избавиться от дублирующих строк.